In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

In [4]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [7]:
# Download data

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('imgs', train=True,
                   transform=transforms.ToTensor()),
    batch_size=64, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('imgs', train=False, transform=transforms.ToTensor()),
    batch_size=64, shuffle=True)

In [42]:
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

In [94]:
class Reshape(nn.Module):
    def __init__(self, shape):
        nn.Module.__init__(self)
        self.shape=shape
    def forward(self,input):
        return input.view(self.shape)

In [142]:
# Create VAE
CODE_SIZE = 100

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 32, 3, stride=3, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(32, 64, 3, stride=2, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1),
            Flatten(),
            nn.ReLU(True),
            nn.Linear(256, 10*2)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(10, 64*2*2),
            nn.ELU(),
            Reshape([-1, 64, 2, 2]),
            nn.ConvTranspose2d(64, 32, 3, stride=2),
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, 5, stride=3, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 1, 2, stride=2, padding=1),
            nn.Sigmoid()
        )


    def reparameterize(self, mu, logvar):
        return torch.randn(10) * logvar + mu

    def forward(self, x):
        encoder_outputs = self.encoder(x)
        mus = encoder_outputs[:, :10]
        logvars = torch.exp(encoder_outputs[:, 10:] / 2)
        decoder_inputs = self.reparameterize(mus, logvars)
        return self.decoder(decoder_inputs), mus, logvars
            


In [145]:
model = VAE()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    # Just skip it !
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), size_average=False)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [146]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [147]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
#             data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(64, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [148]:
!rm -rf results
!mkdir results

In [149]:
EPOCH = 1000 # change me plz!


for epoch in range(1, EPOCH):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 10)
        sample = model.decoder(sample)
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1474: UserWarning: Using a target size (torch.Size([64, 784])) that is different to the input size (torch.Size([64, 1, 28, 28])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Train Epoch: 1 [0/60000 (0%)]	Loss: 560.525391
Train Epoch: 1 [3200/60000 (5%)]	Loss: 280.275269
Train Epoch: 1 [6400/60000 (11%)]	Loss: 206.764450
Train Epoch: 1 [9600/60000 (16%)]	Loss: 209.354111
Train Epoch: 1 [12800/60000 (21%)]	Loss: 197.295303
Train Epoch: 1 [16000/60000 (27%)]	Loss: 178.476410
Train Epoch: 1 [19200/60000 (32%)]	Loss: 169.947891
Train Epoch: 1 [22400/60000 (37%)]	Loss: 164.425430
Train Epoch: 1 [25600/60000 (43%)]	Loss: 151.503052
Train Epoch: 1 [28800/60000 (48%)]	Loss: 148.303543
Train Epoch: 1 [32000/60000 (53%)]	Loss: 140.394806
Train Epoch: 1 [35200/60000 (59%)]	Loss: 139.105865
Train Epoch: 1 [38400/60000 (64%)]	Loss: 141.942795
Train Epoch: 1 [41600/60000 (69%)]	Loss: 130.692764
Train Epoch: 1 [44800/60000 (75%)]	Loss: 127.381721
Train Epoch: 1 [48000/60000 (80%)]	Loss: 120.119553
Train Epoch: 1 [51200/60000 (85%)]	Loss: 124.283577
Train Epoch: 1 [54400/60000 (91%)]	Loss: 125.451134
Train Epoch: 1 [57600/60000 (96%)]	Loss: 117.094398


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1474: UserWarning: Using a target size (torch.Size([32, 784])) that is different to the input size (torch.Size([32, 1, 28, 28])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


====> Epoch: 1 Average loss: 171.7823


/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1474: UserWarning: Using a target size (torch.Size([16, 784])) that is different to the input size (torch.Size([16, 1, 28, 28])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


====> Test set loss: 119.6040
Train Epoch: 2 [0/60000 (0%)]	Loss: 118.442734
Train Epoch: 2 [3200/60000 (5%)]	Loss: 128.174789
Train Epoch: 2 [6400/60000 (11%)]	Loss: 117.932999
Train Epoch: 2 [9600/60000 (16%)]	Loss: 117.622993
Train Epoch: 2 [12800/60000 (21%)]	Loss: 117.002937
Train Epoch: 2 [16000/60000 (27%)]	Loss: 111.900238
Train Epoch: 2 [19200/60000 (32%)]	Loss: 115.047974
Train Epoch: 2 [22400/60000 (37%)]	Loss: 105.013687
Train Epoch: 2 [25600/60000 (43%)]	Loss: 112.599426
Train Epoch: 2 [28800/60000 (48%)]	Loss: 112.883339
Train Epoch: 2 [32000/60000 (53%)]	Loss: 113.249611
Train Epoch: 2 [35200/60000 (59%)]	Loss: 111.091728
Train Epoch: 2 [38400/60000 (64%)]	Loss: 105.245514
Train Epoch: 2 [41600/60000 (69%)]	Loss: 113.539032
Train Epoch: 2 [44800/60000 (75%)]	Loss: 107.482010
Train Epoch: 2 [48000/60000 (80%)]	Loss: 114.335938
Train Epoch: 2 [51200/60000 (85%)]	Loss: 111.485901
Train Epoch: 2 [54400/60000 (91%)]	Loss: 102.319611
Train Epoch: 2 [57600/60000 (96%)]	Loss: 10

In [150]:
torch.save(model, "VAE_weights.pt")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type VAE. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Flatten. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type Reshape. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [153]:
from functools import partial
import pickle

pickle.load = partial(pickle.load, encoding="latin1")
pickle.Unpickler = partial(pickle.Unpickler, encoding="latin1")

vae = torch.load('VAE_weights.pt', map_location=lambda storage, loc: storage, pickle_module=pickle)

In [154]:
vae

VAE(
  (encoder): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(3, 3), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten()
    (7): Linear(in_features=256, out_features=20, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=10, out_features=256, bias=True)
    (1): ELU(alpha=1.0)
    (2): Reshape()
    (3): ConvTranspose2d(64, 32, kernel_size=(3, 3), stride=(2, 2))
    (4): ReLU(inplace)
    (5): ConvTranspose2d(32, 16, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (6): ReLU(inplace)
    (7): ConvTranspose2d(16, 1, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
    (8): Sigmoid()
  )
)